# Training environment information

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        642M         10G        1.2M        2.0G         11G
Swap:            0B          0B          0B


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-4745fba3-8237-26a6-369a-481cf66b3146)


In [3]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz


In [4]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [5]:
!nvidia-smi

Mon Jun 13 15:50:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [7]:
!pip install fastai==2.5.3 -q
!pip freeze

     |████████████████████████████████| 189 kB 5.0 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x39518000 @  0x7fa29659f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 21.0 MB 8.6 MB/s 
     |████████████████████████████████| 23.2 MB 1.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.10.2 which is incompatible.
torchaudio 0.11.0+cu113 requires torc

In [8]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [9]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [10]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Running on COLAB
Mounted at /gdrive


# Data Selection

In [11]:

# Data acquition file to load from dateset folder
# battery_list=["02_1","02_2","02_3","02_4","02_5","02_6","02_7","02_8","02_9"]
battery_list=["02_4","02_5","02_6","02_7","02_8","02_9","03_4","03_5","03_6","03_7","03_8","03_9","05_3","05_4","05_5","05_6","05_7","05_8"]

# Model Training

## Single model Train/Validation split

### Single Model - NO data Augmentation

In [12]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_NO_AUGMENTATION"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=1)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_4', '02_5', '02_6', '02_7', '02_8', '02_9', '03_4', '03_5', '03_6', '03_7', '03_8', '03_9', '05_3', '05_4', '05_5', '05_6', '05_7', '05_8']
model name: Paper_EIS_single_model_NO_AUGMENTATION
dataset row number: 180
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION
df_real shape: (180, 14)
df_img shape: (180, 14)
soc: 100
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_4_100.png
soc: 090
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_4_090.png
soc: 080
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_4_080.png
soc: 070
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/ba

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Valley: 1.45e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.552260,3.460191,0.126984,00:02


epoch,train_loss,valid_loss,accuracy,time
0,3.409481,3.216151,0.111111,00:02
1,3.153735,2.964008,0.126984,00:01
2,3.042181,2.538019,0.126984,00:01
3,2.988706,2.345674,0.174603,00:01
4,2.944555,2.135704,0.285714,00:01
5,2.844972,2.124272,0.317460,00:01
6,2.766246,2.055532,0.333333,00:01
7,2.630303,2.069182,0.333333,00:01
8,2.529889,2.068088,0.333333,00:01
9,2.412875,2.047266,0.412698,00:01


saved filename: Paper_EIS_single_model_NO_AUGMENTATION_1655135698.941734_SAVED.pth
Setting-up type transforms pipelines
Found 210 items
2 datasets of sizes 147,63
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_8_020.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_NO_AUGMENTATION/Paper_EIS_single_model_NO_AUGMENTATION/EIS_BATT02_8_020.png
    applying RegexLabeller gives
      020
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(1)

Final sample: (PILImage mode=RGB size=432x288, Te

In [13]:
score_model(weights_filename,dl)

[[7 2 0 0 0 0 0 0 0 0]
 [0 3 0 1 0 0 0 0 0 0]
 [0 1 3 0 0 0 0 0 0 0]
 [0 0 5 4 0 0 0 0 0 0]
 [0 1 1 2 1 0 0 0 0 0]
 [0 0 0 0 0 7 1 0 0 0]
 [0 0 0 0 0 0 2 1 0 0]
 [0 0 0 0 0 0 1 2 2 0]
 [0 0 0 0 1 0 0 0 8 0]
 [0 0 1 0 0 0 0 0 1 5]]
[[0.77777778 0.22222222 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.75       0.         0.25       0.         0.
  0.         0.         0.         0.        ]
 [0.         0.25       0.75       0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.55555556 0.44444444 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.2        0.2        0.4        0.2        0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.875
  0.125      0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.66666667 0.33333333 0.         0.        ]
 [0.         0.  

learn.validate()


Model accuracy: 0.67
learn.get_preds()


TensorBase(0.6667)

### Single model - Data Augmentation Very low noise

In [14]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_LOW_NOISE"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,NOISE_AMOUNT=1e-12,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_4', '02_5', '02_6', '02_7', '02_8', '02_9', '03_4', '03_5', '03_6', '03_7', '03_8', '03_9', '05_3', '05_4', '05_5', '05_6', '05_7', '05_8']
model name: Paper_EIS_single_model_LOW_NOISE
dataset row number: 180
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE
df_real shape: (180, 14)
df_img shape: (180, 14)
soc: 100
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT02_4_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1300_100.png
augmentation_index: 4
/

Valley: 1.45e-03


epoch,train_loss,valid_loss,accuracy,time
0,2.787553,1.705569,0.389356,00:08


epoch,train_loss,valid_loss,accuracy,time
0,1.643045,0.815297,0.705882,00:08
1,1.419648,0.596310,0.826331,00:08
2,1.265769,0.462183,0.851541,00:08
3,1.095477,0.351221,0.896359,00:08
4,0.986807,0.365474,0.890756,00:08
5,0.881241,0.307422,0.910364,00:08
6,0.794432,0.360953,0.879552,00:08
7,0.723809,0.265791,0.910364,00:08
8,0.647377,0.309296,0.910364,00:08
9,0.587905,0.220335,0.941176,00:08


saved filename: Paper_EIS_single_model_LOW_NOISE_1655136346.981976_SAVED.pth
Setting-up type transforms pipelines
Found 1190 items
2 datasets of sizes 833,357
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1171_090.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_LOW_NOISE/Paper_EIS_single_model_LOW_NOISE/EIS_BATT1171_090.png
    applying RegexLabeller gives
      090
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(8)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(8))


Found 119

In [15]:
score_model(weights_filename,dl)

[[32  0  0  0  0  1  0  0  0  0]
 [ 0 36  0  0  0  0  0  0  0  1]
 [ 0  0 30  2  0  0  0  0  0  1]
 [ 0  0  1 29  0  0  0  0  0  1]
 [ 0  0  0  0 37  0  0  0  0  0]
 [ 0  0  0  0  0 37  0  0  0  0]
 [ 0  0  0  0  0  0 34  0  0  0]
 [ 0  0  0  0  0  0  1 39  1  0]
 [ 0  0  0  0  0  0  1  0 39  0]
 [ 0  0  1  0  0  0  0  0  0 33]]
[[0.96969697 0.         0.         0.         0.         0.03030303
  0.         0.         0.         0.        ]
 [0.         0.97297297 0.         0.         0.         0.
  0.         0.         0.         0.02702703]
 [0.         0.         0.90909091 0.06060606 0.         0.
  0.         0.         0.         0.03030303]
 [0.         0.         0.03225806 0.93548387 0.         0.
  0.         0.         0.         0.03225806]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.        ]
 [0.         0.    

learn.validate()


Model accuracy: 0.97
learn.get_preds()


TensorBase(0.9692)

### Single Model - Data augmentation with realistic AGW noise amount in data acquisition simulation

In [16]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_4', '02_5', '02_6', '02_7', '02_8', '02_9', '03_4', '03_5', '03_6', '03_7', '03_8', '03_9', '05_3', '05_4', '05_5', '05_6', '05_7', '05_8']
model name: Paper_EIS_single_model
dataset row number: 180
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model
df_real shape: (180, 14)
df_img shape: (180, 14)
soc: 100
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT02_4_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1300_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1400_100.png
augmentat

Valley: 1.45e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.063414,2.505923,0.305322,00:07


epoch,train_loss,valid_loss,accuracy,time
0,1.907127,1.271272,0.546219,00:07
1,1.814533,1.093316,0.658263,00:07
2,1.727535,0.947833,0.669468,00:07
3,1.571210,0.762135,0.719888,00:07
4,1.431842,0.717370,0.733894,00:07
5,1.339629,0.619431,0.789916,00:07
6,1.243062,0.646445,0.764706,00:07
7,1.106655,0.525635,0.806723,00:07
8,1.036740,0.572567,0.798319,00:08
9,0.976105,0.724998,0.745098,00:07


saved filename: Paper_EIS_single_model_1655137061.567779_SAVED.pth
Setting-up type transforms pipelines
Found 1190 items
2 datasets of sizes 833,357
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1171_090.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model/Paper_EIS_single_model/EIS_BATT1171_090.png
    applying RegexLabeller gives
      090
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(8)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(8))


Found 1190 items
2 datasets of sizes 833,357
Setting up Pip

In [17]:
score_model(weights_filename,dl)

[[32  1  0  0  0  0  0  0  0  0]
 [ 0 34  1  0  1  0  0  0  0  1]
 [ 0  4 23  4  0  0  0  0  0  2]
 [ 0  0  1 28  0  0  0  0  0  2]
 [ 0  0  0  2 35  0  0  0  0  0]
 [ 0  0  0  0  0 37  0  0  0  0]
 [ 0  0  0  0  0  0 31  3  0  0]
 [ 0  0  0  0  0  0  7 31  3  0]
 [ 0  0  0  0  0  0  2  2 36  0]
 [ 0  2  2  1  0  0  0  0  0 29]]
[[0.96969697 0.03030303 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.91891892 0.02702703 0.         0.02702703 0.
  0.         0.         0.         0.02702703]
 [0.         0.12121212 0.6969697  0.12121212 0.         0.
  0.         0.         0.         0.06060606]
 [0.         0.         0.03225806 0.90322581 0.         0.
  0.         0.         0.         0.06451613]
 [0.         0.         0.         0.05405405 0.94594595 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.        ]
 [0.         0.         0. 

learn.validate()


Model accuracy: 0.89
learn.get_preds()


TensorBase(0.8852)